<a href="https://colab.research.google.com/github/BenUCL/Reef-acoustics-and-AI/blob/main/Code/Feature_extraction_with_pretrained_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connect your Google drive so you can access uploaded data:

Download the pretrained CNN (VGGish) and required packages. The smoke test code block should return 'Looks good to me!' right at the bottom.

In [8]:
### Change patsh if you structure folders

# Path to the location where your audio file are stored:
audio_dir = "../../data/HD_audio"

# Path to folder containing vggish setup files and 'AudiosetAnalysis' downloaded from sarebs supplementary
vggish_files = "Audioset"

# Output folder for results:
output_folder_1min = "../Results"


In [2]:
import os

#Navigate to the folder containing setup files, including AudiosetAnalysis downloaded from sarebs supplementary
os.chdir(vggish_files) 

In [3]:
from datetime import datetime
from datetime import timedelta
import pandas as pd
from AudiosetAnalysis_aug import AudiosetAnalysis

Instructions for updating:
non-resource variables are not supported in the long term


Set up the Audioset analysis. 

RUN THIS BLOCK ONCE PER SESSION, otherwise it will error

In [4]:
an = AudiosetAnalysis()
an.setup()

INFO:tensorflow:Restoring parameters from vggish_model.ckpt


2023-11-29 10:04:39.036971: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-11-29 10:04:39.036997: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-11-29 10:04:39.037007: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-11-29 10:04:39.037072: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-29 10:04:39.037109: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/simbashi/Developer/miniforge3/envs/reefsound_tensorflow/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1697: UserWarning: `layer.apply` is deprecated 

This code block extracts pretrained CNN (VGGish) features from each example audio file in your GDrive and saves a timestamped CSV to your results folder in the GDrive.

In [9]:
#select files
all_fs = os.listdir(audio_dir) #list of all files in directory
audio_fs = [f for f in all_fs if '.wav' in f.lower() or '.mp3' in f.lower()] #list of all audio files in dir: .wav or .mps

#initiate empty dataframes to save results
results_df_1sec = pd.DataFrame()
results_df_1min = pd.DataFrame() 

# Feature extraction loop
for f in audio_fs:
    """This loop takes the current filename, rips the timestamp, appends the corresponding length of time being analysed
    additively to a new name, calculates VGGish features from each 0.96s chunk, averages these for each 1min file and saves 
    the results to a csv in your GDrive"""
    path = os.path.join(audio_dir, f)
    print(f) #print file name
    
    #extract timestamp from filename - adjust this if using your own naming convention
    t2 = f.split(".")[1][0:4]
    t1 = f.split(".")[3]
    t = t1+t2+'00'
    recording_start_time = pd.to_datetime(t, format='%y%m%d%H%M%S') 
    slice_time = recording_start_time - timedelta(milliseconds=960)
    mean_slice_time = recording_start_time - timedelta(minutes=1)
    
    #calculate feature values
    # results = an.analyse_audio(path)
    aug_count = 0
    for results in an.analyse_audio(path):
    
        # Uncomment for 0.96s results:
        # r1sec = results['raw_audioset_feats_960ms']
        # for count, r1sec in enumerate(r1sec):
        #     slice_time = slice_time + timedelta(milliseconds=960)
        #     string_time = slice_time.strftime('%H.%M.%S.%f')[:-4]
        #     result_name = f[:-4] + 'T' + string_time + '.wav'
        #     #result_name = f[:-4]+'T'+str(count+1)+'.wav' #use this line if not using ST timestamped files
        #     results_df_1sec[result_name] = pd.Series(results['raw_audioset_feats_960ms'][count])

        #Save 1min results:
        r1min = results['raw_audioset_feats_59520ms']
        for count, r1min in enumerate(r1min):
            #store the timestamp
            mean_slice_time = mean_slice_time + timedelta(minutes=1)
            string_time = mean_slice_time.strftime('%H.%M.%S.%f')[:-4]
            result_name = f[:-4] + 'T' + string_time + "A" + str(aug_count) + '.wav'
            #result_name = f[:-4]+'T'+str(count+1)+'.wav' #use this line if not using ST timestamped files
            results_df_1min[result_name] = pd.Series(results['raw_audioset_feats_59520ms'][count])
        aug_count += 1


#save a timestamped csv with 1min results
now = datetime.now()
time_now = now.strftime("%H.%M.%S")
results_df_1min.to_csv(output_folder_1min + '/pretrained_CNN_features_' + time_now + '.csv')
print("it's done.results is saved in {}".format(output_folder_1min + '/pretrained_CNN_features_' + time_now + '.csv'))
# print("it's done")

SaF1.1733D.1677983769.180827.2.16.wav
AudiosetAnalysis: Calculating log mel spectrogram for ../../data/HD_audio/SaF1.1733D.1677983769.180827.2.16.wav
SaN11.0940D.1678278701.180906.5.59.wav
AudiosetAnalysis: Calculating log mel spectrogram for ../../data/HD_audio/SaN11.0940D.1678278701.180906.5.59.wav
BoF4.2333H.1677983769.180830.5.54.wav
AudiosetAnalysis: Calculating log mel spectrogram for ../../data/HD_audio/BoF4.2333H.1677983769.180830.5.54.wav
BoF3.1205D.805322778.180829.2.9.wav
AudiosetAnalysis: Calculating log mel spectrogram for ../../data/HD_audio/BoF3.1205D.805322778.180829.2.9.wav
SaF1.2333D.1677983769.180827.2.15.wav
AudiosetAnalysis: Calculating log mel spectrogram for ../../data/HD_audio/SaF1.2333D.1677983769.180827.2.15.wav
BoF5.0532H.1677983769.180831.3.37.wav
AudiosetAnalysis: Calculating log mel spectrogram for ../../data/HD_audio/BoF5.0532H.1677983769.180831.3.37.wav
BoN11.1200D.1678278701.180906.5.20.wav
AudiosetAnalysis: Calculating log mel spectrogram for ../../dat

/var/folders/yz/_wj_p5r56c7ggl53_9wppwl80000gn/T/ipykernel_55832/1027907342.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results_df_1min[result_name] = pd.Series(results['raw_audioset_feats_59520ms'][count])


BaF5.2332H.671907872.180831.4.39.wav
AudiosetAnalysis: Calculating log mel spectrogram for ../../data/HD_audio/BaF5.2332H.671907872.180831.4.39.wav


/var/folders/yz/_wj_p5r56c7ggl53_9wppwl80000gn/T/ipykernel_55832/1027907342.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results_df_1min[result_name] = pd.Series(results['raw_audioset_feats_59520ms'][count])
/var/folders/yz/_wj_p5r56c7ggl53_9wppwl80000gn/T/ipykernel_55832/1027907342.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results_df_1min[result_name] = pd.Series(results['raw_audioset_feats_59520ms'][count])
/var/folders/yz/_wj_p5r56c7ggl53_9wppwl80000gn/T/ipykernel_55832/1027907342.py:47: PerformanceWarning:

BoF2.0930D.1678278701.180828.4.56.wav
AudiosetAnalysis: Calculating log mel spectrogram for ../../data/HD_audio/BoF2.0930D.1678278701.180828.4.56.wav


...

SaN10.1345D.1677983769.180905.1.14.wav
AudiosetAnalysis: Calculating log mel spectrogram for ../../data/HD_audio/SaN10.1345D.1677983769.180905.1.14.wav


/var/folders/yz/_wj_p5r56c7ggl53_9wppwl80000gn/T/ipykernel_55832/1027907342.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results_df_1min[result_name] = pd.Series(results['raw_audioset_feats_59520ms'][count])
/var/folders/yz/_wj_p5r56c7ggl53_9wppwl80000gn/T/ipykernel_55832/1027907342.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results_df_1min[result_name] = pd.Series(results['raw_audioset_feats_59520ms'][count])


BaF5.1732H.671907872.180831.4.31.wav
AudiosetAnalysis: Calculating log mel spectrogram for ../../data/HD_audio/BaF5.1732H.671907872.180831.4.31.wav


/var/folders/yz/_wj_p5r56c7ggl53_9wppwl80000gn/T/ipykernel_55832/1027907342.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results_df_1min[result_name] = pd.Series(results['raw_audioset_feats_59520ms'][count])
/var/folders/yz/_wj_p5r56c7ggl53_9wppwl80000gn/T/ipykernel_55832/1027907342.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results_df_1min[result_name] = pd.Series(results['raw_audioset_feats_59520ms'][count])


it's done.results is saved in ../Results/pretrained_CNN_features_10.14.04.csv
